# Yolo

### environnement pour Yolo :

```python
pip install ultralytics
```

### Format attendu par Yolo :

```python
datasets/
└── pyro_sdis/
    ├── images/
    │   ├── train/  (vos images d'entrainement)
    │   └── val/    (vos images de validation)
    └── labels/
        ├── train/  (fichiers .txt correspondants)
        └── val/    (fichiers .txt correspondants)
```

**Format** :        
* Pour chaque image image_01.jpg il faut un fichier image_01.txt      
* Si l'image contient du feu, le fichier contient une ligne par feu : 0 x_center y_center width height            
  * Les valeurs doivent être noramlisées entre 0 et 1 par rapport à la taille de l'image
  * Si l'image est NO FIRE, fichier .txt doit être vide
  


le code ci dessous :        

### Nettoyage
suppression des éventuels dossier ./pyro-sdis
### Création de l'arborescence :
``os.makedirs``
* créer l'arborescence exigé de dossiers vides
  * séparer images.jpg et labels.txt
  * créer sous-dossier train et val dans chacun de ces dossiers
### Telechargement de la configuration :
``hf_hub_download ``
* Va chercher le fichier data.yaml sur le serveur
  * Ce fichier sert à définir le dataset pour yolo
### Comparé aux anciens modèles (inception)
* Yolo lit directement les fichiers sur disque
* Yolo redimensionne les images automatiquement pendant l'entrainement
  * suppr de ``transform.Compose``
* Yolo gère les déséquilibre via "Mosaic augmentation" en interne
  * suppr de ``WeightRandomSampler``
* Les batch ne sont plus manuellement chargé en RAM
  * surpp de ``DataLoader``


| Epoch | Time (s) | Train Box Loss | Train Cls Loss | Precision (B) | Recall (B) | mAP50 (B) | mAP50-95 (B) | Val Box Loss | Val Cls Loss |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **1** | 782.70 | 1.7988 | 1.6933 | 0.585 | 0.579 | 0.579 | 0.299 | 1.6125 | 1.5801 |
| **2** | 1528.89 | 1.6766 | 1.5158 | 0.641 | 0.645 | 0.669 | 0.377 | 1.4679 | 1.3872 |
| **3** | 2263.62 | 1.6261 | 1.4380 | 0.645 | 0.671 | 0.689 | 0.389 | 1.4195 | 1.3567 |
| **4** | 2999.97 | 1.5686 | 1.3722 | 0.672 | 0.674 | 0.710 | 0.417 | 1.3784 | 1.3179 |
| **5** | 3734.68 | 1.5248 | 1.3146 | 0.688 | 0.692 | 0.728 | 0.430 | 1.3621 | 1.2619 |
| **6** | 4470.00 | 1.4922 | 1.2629 | 0.700 | 0.721 | 0.756 | 0.456 | 1.3233 | 1.2036 |
| **7** | 5214.70 | 1.4491 | 1.2101 | 0.703 | 0.732 | 0.762 | 0.469 | 1.2939 | 1.1702 |
| **8** | 5959.95 | 1.4287 | 1.1875 | 0.723 | 0.726 | 0.777 | 0.480 | 1.2661 | 1.1424 |
| **9** | 6708.57 | 1.4029 | 1.1518 | 0.724 | 0.731 | 0.783 | 0.488 | 1.2620 | 1.1311 |
| **10** | 7455.81 | 1.3796 | 1.1354 | 0.736 | 0.744 | 0.796 | 0.504 | 1.2405 | 1.1035 |
| **11** | 8328.94 | 1.3691 | 1.1439 | 0.738 | 0.745 | 0.796 | 0.501 | 1.2382 | 1.0839 |
| **12** | 9074.00 | 1.3611 | 1.1205 | 0.745 | 0.753 | 0.807 | 0.513 | 1.2299 | 1.0558 |
| **13** | 9799.08 | 1.3307 | 1.0929 | 0.743 | 0.768 | 0.812 | 0.519 | 1.2163 | 1.0422 |
| **14** | 10531.1 | 1.3196 | 1.0649 | 0.748 | 0.762 | 0.812 | 0.523 | 1.2063 | 1.0186 |
| **15** | 11265.5 | 1.2987 | 1.0452 | 0.751 | 0.771 | 0.822 | 0.530 | 1.1950 | 1.0127 |
| **16** | 12011.9 | 1.2872 | 1.0229 | 0.758 | 0.771 | 0.826 | 0.536 | 1.1890 | 0.9925 |
| **17** | 12751.2 | 1.2654 | 1.0028 | 0.757 | 0.778 | 0.826 | 0.536 | 1.1861 | 0.9853 |
| **18** | 13488.9 | 1.2516 | 0.9800 | 0.761 | 0.781 | 0.830 | 0.541 | 1.1857 | 0.9812 |
| **19** | 14230.5 | 1.2355 | 0.9680 | 0.773 | 0.770 | 0.832 | 0.544 | 1.1793 | 0.9741 |
| **20** | 14986.2 | 1.2260 | 0.9397 | **0.773** | **0.769** | **0.833** | **0.546** | **1.1781** | **0.9731** |

![results.png](attachment:results.png)
VRAI POSITIFS : 85 %            
FAUX POSITIFS : 15 %
![confusion_matrix_normalized.png](attachment:confusion_matrix_normalized.png)
![confusion_matrix.png](attachment:confusion_matrix.png)

In [ ]:
'''import os
import shutil
import yaml
from datasets import load_dataset
from tqdm import tqdm

# --- CONFIGURATION ---
REPO_ID = "pyronear/pyro-sdis"
# chemin ABSOLU
BASE_DIR = os.getcwd()
OUTPUT_DIR = os.path.join(BASE_DIR, "pyro-sdis")
IMAGE_DIR = os.path.join(OUTPUT_DIR, "images")
LABEL_DIR = os.path.join(OUTPUT_DIR, "labels")

print(f"Le dataset sera installé ici : {OUTPUT_DIR}")

# 1. NETTOYAGE
if os.path.exists(OUTPUT_DIR):
    print("   Suppression de l'ancien dossier...")
    shutil.rmtree(OUTPUT_DIR)

# 2. CRÉATION ARBORESCENCE
for split in ["train", "val"]:
    os.makedirs(os.path.join(IMAGE_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(LABEL_DIR, split), exist_ok=True)

# 3. CHARGEMENT ET EXTRACTION
print("⬇Chargement du dataset depuis Hugging Face...")
dataset = load_dataset(REPO_ID)

def save_clean_yolo_format(dataset_split, split):
    print(f" Traitement du split : {split}")

    for example in tqdm(dataset_split):
        # --- A. IMAGE ---
        if "image_name" in example and example["image_name"]:
            image_name = example["image_name"]
        else:
            continue # On saute si pas de nom

        image = example["image"]
        image.save(os.path.join(IMAGE_DIR, split, image_name))

        # --- B. LABEL (LA CORRECTION EST ICI) ---
        # on corrige la classe à 0, on écrit.

        annotations = example["annotations"] # C'est une chaîne de caractères
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(LABEL_DIR, split, label_name)

        # Si l'image n'a pas d'annotation (vide), on crée un fichier vide
        if not annotations or len(annotations.strip()) == 0:
            open(label_path, 'w').close()
            continue

        # Si il y a des annotations, on force la classe à 0
        clean_lines = []
        for line in annotations.strip().split('\n'):
            parts = line.strip().split()
            if len(parts) >= 5: # Sécurité pour vérifier que la ligne est valide
                # parts[0] est la classe. On la REMPLACE par "0"
                parts[0] = "0"
                clean_lines.append(" ".join(parts))

        with open(label_path, "w") as f:
            f.write("\n".join(clean_lines))

# Lancement de l'extraction
save_clean_yolo_format(dataset["train"], "train")
save_clean_yolo_format(dataset["val"], "val")

# 4. GÉNÉRATION DU DATA.YAML (DEUXIÈME CORRECTION)

yaml_content = {
    'path': OUTPUT_DIR,       # Chemin ABSOLU vers le dossier pyro-sdis
    'train': 'images/train',  # Relatif à path
    'val': 'images/val',      # Relatif à path
    'names': {
        0: 'fire'
    },
    'nc': 1
}

yaml_path = os.path.join(BASE_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False)

print("\nTERMINÉ !")
print(f"Fichier de config généré : {yaml_path}")
'''

c:\Users\axelv\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Le dataset sera installé ici : c:\Users\axelv\Documents\Jedha\Projet\Projet_Fire_dection\Fire_detection\models\Model_03_Yolo_960\pyro-sdis
⬇Chargement du dataset depuis Hugging Face...
 Traitement du split : train


100%|██████████| 29537/29537 [05:09<00:00, 95.36it/s] 


 Traitement du split : val


100%|██████████| 4099/4099 [01:00<00:00, 68.10it/s]


TERMINÉ !
Fichier de config généré : c:\Users\axelv\Documents\Jedha\Projet\Projet_Fire_dection\Fire_detection\models\Model_03_Yolo_960\data.yaml


In [ ]:
'''import os
import shutil
import random
from pathlib import Path

# --- CONFIGURATION ---
# Mets ici le chemin vers ton dossier PRINCIPAL qui contient 'images' et 'labels'
dataset_path = Path(r"C:\Users\axelv\Documents\Jedha\Projet\Projet_Fire_dection\Fire_detection\models\Model_03_Yolo_960\pyro-sdis")
split_ratio = 0.80  # 80% pour l'entrainement, 20% pour la validation

# --- DÉBUT DU SCRIPT ---
print("--- Démarrage du mélange ---")

# 1. Identifier tous les dossiers
img_train = dataset_path / "images" / "train"
img_val = dataset_path / "images" / "val"
lbl_train = dataset_path / "labels" / "train"
lbl_val = dataset_path / "labels" / "val"

# Créer les dossiers s'ils n'existent pas (sécurité)
for p in [img_train, img_val, lbl_train, lbl_val]:
    p.mkdir(parents=True, exist_ok=True)

# 2. Rassembler TOUTES les images (Train + Val actuel) dans une liste
all_images = []
# On regarde dans train
for f in os.listdir(img_train):
    if f.endswith(('.jpg', '.png', '.jpeg')):
        all_images.append((img_train / f, lbl_train / f.replace(f.split('.')[-1], 'txt')))
# On regarde dans val
for f in os.listdir(img_val):
    if f.endswith(('.jpg', '.png', '.jpeg')):
        all_images.append((img_val / f, lbl_val / f.replace(f.split('.')[-1], 'txt')))

print(f"Total images trouvées : {len(all_images)}")

# 3. MELANGER (Le plus important !)
random.seed(42) # Pour que ce soit reproductible
random.shuffle(all_images)

# 4. Calculer la séparation
split_index = int(len(all_images) * split_ratio)
train_set = all_images[:split_index]
val_set = all_images[split_index:]

print(f"Nouvelle répartition -> Train: {len(train_set)} | Val: {len(val_set)}")

# 5. Déplacer les fichiers
def move_files(file_list, dest_img_folder, dest_lbl_folder):
    for img_path, lbl_path in file_list:
        # Déplacer l'image
        if img_path.exists():
            shutil.move(str(img_path), str(dest_img_folder / img_path.name))

        # Déplacer le label correspondant (s'il existe)
        if lbl_path.exists():
            shutil.move(str(lbl_path), str(dest_lbl_folder / lbl_path.name))

print("Déplacement des fichiers en cours...")
# On vide tout vers les bonnes destinations
move_files(train_set, img_train, lbl_train)
move_files(val_set, img_val, lbl_val)

print("--- Terminé ! Ton dataset est mélangé. ---")
print("N'oublie pas de supprimer les fichiers *.cache dans tes dossiers labels avant de relancer YOLO !")'''

--- Démarrage du mélange ---
Total images trouvées : 33636
Nouvelle répartition -> Train: 26908 | Val: 6728
Déplacement des fichiers en cours...
--- Terminé ! Ton dataset est mélangé. ---
N'oublie pas de supprimer les fichiers *.cache dans tes dossiers labels avant de relancer YOLO !


In [ ]:
!yolo detect train model="TON_CHEMIN_ICI/weights/best.pt" data="data.yaml" epochs=30 lr0=0.0001 lrf=0.01 warmup_epochs=0 freeze=0 batch=16 project="Pyro_Yolo_960" name="phase03_FORCED"

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/bin/yolo", line 7, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/ultralytics/cfg/__init__.py", line 955, in entrypoint
    model = YOLO(model, task=task)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/ultralytics/models/yolo/model.py", line 81, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/ultralytics/engine/model.py", line 149, in __init__
    self._load(model, task=task)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/ultralytics/engine/model.py", line 288, in _load
    self.model, self.ckpt = load_checkpoint(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zeus/miniconda3/envs/cloudspace/lib/pyt

In [ ]:
from ultralytics import YOLO
import os

def main_step2():
    path_to_step1_model = "/teamspace/studios/this_studio/Pyro_Yolo_960/phase02_neck_09/weights/best.pt"
    # Vérification de sécurité
    if not os.path.exists(path_to_step1_model):
        print(f"ERREUR : Le fichier n'existe pas ici : {path_to_step1_model}")
        print("Vérifie que l'entraînement précédent a bien fini et créé le dossier 'weights'.")
        return

    print(f"--- Chargement du modèle de la Phase 1 : {path_to_step1_model} ---")

    # 2. On charge TON modèle entraîné (et pas yolov8s.pt !)
    model = YOLO(path_to_step1_model)

    # 3. Lancement de l'entraînement Phase 2
    model.train(
        data='data.yaml',
        epochs=50,            # On laisse le temps au "Cou" de s'adapter

        # --- LE REGLAGE MAGIQUE ---
        freeze=0,             # On gèle le Backbone (0-9). Le Neck (10-21) et la Head (22) apprennent.

        lr0=0.0001,             # IOn divise la vitesse par 2 (0.01 à 0.005)
        cos_lr=True,
        # --- Tes réglages optimisés ---
        mosaic=1.0,
        hsv_v=0.4,
        fliplr=0.5,
        batch=32,
        workers=8,
        cache=False,
        imgsz=960,
        resume=False,
        warmup_epochs=0,
        project='Pyro_Yolo_960',
        name='phase03_Backdown_01',
        exist_ok=True
    )

    print("--- Phase 3 Terminée ! ---")

if __name__ == '__main__':
    main_step2()

--- Chargement du modèle de la Phase 1 : /teamspace/studios/this_studio/Pyro_Yolo_960/phase02_neck_09/weights/best.pt ---
Ultralytics 8.3.229 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L40S, 45709MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/teamspace/studios/this_studio/Pyro_Yolo_960/phase02_neck_09/weights/best.pt, momentum=0.937, mosai

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.0 ms, read: 812.2±1130.0 MB/s, size: 88.2 KB)
train: Scanning /teamspace/studios/this_studio/pyro-sdis/labels/train.cache... 26908 images, 4400 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 26908/26908 51.5Mit/s 0.0s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 956.6±182.2 MB/s, size: 80.7 KB)
val: Scanning /teamspace/studios/this_studio/pyro-sdis/labels/val.cache... 6728 images, 1099 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 6728/6728 9.7Mit/s 0.0s0s
Plotting labels to /teamspace/studios/this_studio/Pyro_Yolo_960/phase03_Backdown_01/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image siz

KeyboardInterrupt: 

In [ ]:
!yolo detect train model="/teamspace/studios/this_studio/Pyro_Yolo_960/phase02_neck_09/weights/best.pt" data="data.yaml" epochs=30 lr0=0.0001 lrf=0.01 cos_lr=True warmup_epochs=0 freeze=0 batch=16 project="Pyro_Yolo_960" name="phase04_cosine_CLI_FINAL"

Ultralytics 8.3.229 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA L40S, 45709MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/teamspace/studios/this_studio/Pyro_Yolo_960/phase02_neck_09/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=phase04_cosine_CLI_FINAL, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

In [ ]:
import os
import time

print("L'entraînement est terminé ! J'attends 60 secondes pour être sûr que les fichiers sont bien sauvegardés...")
time.sleep(60) # On laisse le temps au disque dur de souffler

print("Poutoux !")
# Cette commande dit à Linux d'éteindre la machine immédiatement
os.system("sudo shutdown now")

⏳ L'entraînement est fini ! J'attends 60 secondes pour être sûr que les fichiers sont bien sauvegardés...
